# Проект для «Викишоп»

<b>Цели проекта:</b>  
Обучить модель классифицировать комментарии на позитивные и негативные и построить модель со значением метрики качества F1 не меньше 0.75.
  
<b>Задачи проекта:</b>
- Загрузить данные и их леммизировать.
- Обучить разные модели с различными гиперпараметрами.
- Проверить данные на тестовой выборке и сделать выводы.

<b>Данные:</b>  
Данные с текстом комментариев. Данные находятся в файле `toxic_comments.csv`.   
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# импортировать библиотеки
import pandas as pd
import numpy
from tqdm import tqdm
import time

# лемматизация
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer

# регулярные выражения
import re

# для нового мешка слов с учётом стоп-слов
from nltk.corpus import stopwords
import nltk

# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# работа с моделями
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline

In [2]:
# записать файл в data
try: 
    data = pd.read_csv('/datasets/toxic_comments.csv', sep=',')
except:
    data = pd.read_csv('toxic_comments.csv', sep=',')

In [3]:
# открыть файл
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
# изучить файл: метод info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
# изучить файл: метод describe()
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Пропусков в данных не обнаружено. Стобец <code>Unnamed: 0</code> не несёт ценности, его можно удалить.

In [6]:
# удалить столбец
data = data.drop('Unnamed: 0', axis=1)

In [7]:
# проверить, что столбце удалён
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создадим корпус постов. Преобразуем столбец <code>text</code> в список текстов. Переводить тексты в стандартный для Python формат, кодировку Unicode U, не будем, так как тексты на английском. Это может привести к падению ядра из-за увеличения объема занимаемой памяти.

In [8]:
# создать корпус постов
corpus = data['text'].values

Проведём лемматизацию текста (приведём слова к начальной форме — лемме). А также от лишних символов очистим текст  регулярными выражениями.

In [9]:
def lemmatize_with_pos(text):
    lemmatizer = WordNetLemmatizer()
    text = re.sub('[^a-zA-Z]+', ' ', text) 
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    lemmatized_words = []
    for word, tag in tagged_words:
        if tag.startswith('NN'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('R'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            lemmatized_words.append(word)
    return ' '.join(lemmatized_words)

In [10]:
# применить функцию
for i in tqdm(range(len(corpus))):
    corpus[i] = lemmatize_with_pos(corpus[i])

100%|██████████| 159292/159292 [09:43<00:00, 273.12it/s]


In [11]:
# записать данные в новый фрейм
new_data = pd.DataFrame(corpus)
# добавить новый столбец
data['lemm_text'] = new_data[0]
data.head()

,text,toxic,lemm_text
0,Explanation Why the edits make under my userna...,0,Explanation Why the edits make under my userna...
1,D aww He match this background colour I m seem...,0,D aww He match this background colour I m seem...
2,Hey man I m really not try to edit war It s ju...,0,Hey man I m really not try to edit war It s ju...
3,More I can t make any real suggestion on impro...,0,More I can t make any real suggestion on impro...
4,You sir be my hero Any chance you remember wha...,0,You sir be my hero Any chance you remember wha...


Получив новый столбец после лемматизации и очищения от лишних символов, можем разделить данные на две выборки в соотношении 4:1.

In [12]:
# разделить данные на выборки
train, test = train_test_split(data, test_size=0.2, random_state=12345)

In [13]:
# разделить обучающую выборку на признаки (features) и целевую переменную (target)
features_train = train.drop('toxic', axis=1)
target_train = train['toxic']

In [14]:
# разделить тестовую выборку на признаки (features) и целевую переменную (target)
features_test = test.drop('toxic', axis=1)
target_test = test['toxic']

In [15]:
# стоп-слова
try:
    nltk.download('stopwords')
except:
    pass
# объявить набор стоп-слов 
try:
    stopwords = set(stopwords.words('english'))
except:
    pass

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Итоги

1. Открыт и изучен файл методами info(), describe().
2. Удалён столбец <code>Unnamed: 0</code>.
3. Проведена лемматизация и очистка данных.
4. Данные разделены на выборки в соотношении 4:1.
5. Объявлен набор стоп-слов.

## Обучение

In [16]:
# посчитать величину TF-IDF для обучающей выборки
count_tf_idf_train = TfidfVectorizer(stop_words=list(stopwords))
tf_idf_train = count_tf_idf_train.fit_transform(train.lemm_text)
display(tf_idf_train.shape)

# посчитать величину TF-IDF для тестовой выборки
count_tf_idf_test = TfidfVectorizer(stop_words=stopwords)
tf_idf_test = count_tf_idf_train.transform(test.lemm_text)
display(tf_idf_test.shape)

(127433, 140645)

(31859, 140645)

### LogisticRegression

In [17]:
# определить Pipeline для LogisticRegression
lr_pipeline = Pipeline([
    ('lr', LogisticRegression(solver='liblinear'))])

In [18]:
# список параметров для подбора
parameters = {
    'lr__C': [0.1, 1, 10]
}

In [19]:
%%time
start = time.time()

# обучить модель LogisticRegression с кросс-валидацией и подбором параметров
lr_grid = GridSearchCV(lr_pipeline, param_grid=parameters, cv=3)
lr_grid.fit(tf_idf_train, target_train)
lr_score = str(lr_grid.best_score_)
print('LogisticRegression best score: ' + lr_score)

end = time.time()
lr_time = end-start

LogisticRegression best score: 0.9563927767016547
CPU times: user 46.6 s, sys: 48.3 s, total: 1min 34s
Wall time: 1min 34s


### CatBoostClassifier

In [20]:
cat_vars = [var for var in features_train.columns if features_train[var].dtype == "O"]

In [21]:
# определить Pipeline для CatBoostClassifier
catboost_pipeline = Pipeline([
    ('catboost', CatBoostClassifier(cat_features = cat_vars, silent=True))
])

In [22]:
# чисок параметров для подбора
parameters = {
    'catboost__depth': [4, 6, 8]
}

In [23]:
%%time
start = time.time()
# обучить модели CatBoostClassifier с кросс-валидацией и подбором параметров
catboost_grid = GridSearchCV(catboost_pipeline, param_grid=parameters, cv=3)
catboost_grid.fit(features_train, target_train)
cat_score = str(catboost_grid.best_score_)
print('CatBoostClassifier best score: ' + cat_score)

end = time.time()
cat_time = end-start

CatBoostClassifier best score: 0.8990214473521801
CPU times: user 7min 54s, sys: 3.92 s, total: 7min 58s
Wall time: 8min 11s


### Лучшая модель

In [24]:
# напечатать лучшие параметры
lr_params = lr_grid.best_params_
lr_params

{'lr__C': 10}

In [25]:
# создать таблицу для сравнения результатов
data_time = [{'Модель': 'LogisticRegression', 'Время работы, сек.': lr_time, 'score': lr_score},
            {'Модель': 'CatBoostClassifier', 'Время работы, сек.': cat_time, 'score': cat_score}]

dframe = pd.DataFrame(data_time, columns =['Модель', 'Время работы, сек.', 'score'])
dframe

,Модель,"Время работы, сек.",score
0,LogisticRegression,94.988488,0.9563927767016547
1,CatBoostClassifier,491.090554,0.8990214473521801


Лучшей моделью и по времени работы, и по score является LogisticRegression.

In [26]:
# задать алгоритм для модели
lr_pipeline = Pipeline([
    ('lr', LogisticRegression(solver='liblinear', C=10))])
# обучить модель
lr_pipeline.fit(tf_idf_train, target_train)

Pipeline(steps=[('lr', LogisticRegression(C=10, solver='liblinear'))])

In [27]:
# предсказания
pred_lr = lr_pipeline.predict(tf_idf_test)
# посчитать и напечатать f1
f1_lr = f1_score(target_test, pred_lr)
print('F1: {:.2f}'.format(f1_lr))

F1: 0.78


## Выводы

1. Открыт и изучен файл методами info(), describe().
2. Удалён столбец <code>Unnamed: 0</code>.
3. Проведена лемматизация и очистка данных.
4. Данные разделены на выборки в соотношении 4:1.
5. Объявлен набор стоп-слов.
6. Произведено обучение на LogisticRegression и CatBoostRegressor.
7. Посчитаны метрики F1 для каждой модели.
  
Модель LogisticRegression справилась лучше (по результатам F1).